
Email

This is an update on Dec C-NOS Actual Report. Please feel free to contact me if you have any questions.
 - As Japan MO total, Dec C-NOS devliered OO IYA with OOO $MM.
 - With this, 1H to Date C-NOS is OOO IYA with OOOO $MM which is -Opt vs. 1H target (OOO IYA).
     - [By Category] Over-delivery in Baby (+4% vs. 1H Target), Hair (+1%), under-delivery in Laundry (-9%), FE (-10%),   
                       Dish (-6%), Air (-14%),  Fem (-16%), Shave (-10%), Oral (-31%), Braun (-1%), POC (-40%).
     - [By Team] Under-delivery in Div1 (-8%), Div2 (-6%), D-com (-12%), Electro (-8%).
     - [By Top Customer] Over-delivery in Rakuten (+2%), TSURUHA (0%), under-delivery in WELCIA (-10%), CAINZ (-12%), MCC 
                          (-4%), SUGI (-6%), AEON (-5%), COSMOS (-21%), COSTCO (-18%), DONKI (-18%). 


Question
1. 1st Table 
  (SQL) Create SQL to show By Category 1H $MM, 1H IYA (Add Total)
  (Prompt) Write [By Category] part
2. 2nd Table
  (SQL) Create SQL to show By Team 1H $MM, 1H IYA (Add Total)
  (Prompt) Write [By team] part
3. 3rd Table
  (SQL) Create SQL to show By Top Customer 1H $MM, 1H IYA (Add Total)
  (Prompt) Write [By Top Customer] part
 

In [0]:
%sql
-- Category --

-- 中間テーブル(CTE)を作成し、FY2525 1H と 2025M07-12 を UNION する

WITH base_cnos AS (
  -- 中間テーブル：C-NOSだけを抽出しておく
  SELECT
    category,
    period,
    month,
    CAST(forecast_act_est AS INT) AS cnos,     -- 今年C-NOS（整数）
    CAST(ya_actual        AS INT) AS cnos_ya   -- 昨年C-NOS（整数）
  FROM hive_metastore.japan_gold.cnos_vw
  WHERE account = 'C-NOS'
),

fy2525_1h AS (
  -- FY2525 1H のカテゴリー別＋Total
  SELECT
    'FY2525 1H' AS term,                                 -- 期間ラベル
    CASE 
      WHEN GROUPING(category) = 1 THEN 'Total'           -- 合計行は 'Total'
      ELSE category
    END AS category,
    -- C-NOS IYA（今年 ÷ 昨年 × 100、小数点1桁）
    ROUND(
      CASE 
        WHEN SUM(cnos_ya) = 0 THEN NULL                  -- 昨年ゼロはNULL
        ELSE 100.0 * SUM(cnos) / SUM(cnos_ya)
      END,
      1
    ) AS cnos_iya,
    -- C-NOS $MM（今年）：値 / 150.1 / 1000 / 1000、小数点1桁
    ROUND(
      SUM(cnos) / 150.1 / 1000.0 / 1000.0,
      1
    ) AS cnos_mm
  FROM base_cnos
  WHERE period = 'FY2525 1H'
  GROUP BY
    GROUPING SETS (
      (category),   -- カテゴリー別
      ()            -- 全カテゴリー合計
    )
),

m0712 AS (
  -- 2025M07〜2025M12 のカテゴリー別＋Total
  SELECT
    '2025M07-12' AS term,                                -- 期間ラベル
    CASE 
      WHEN GROUPING(category) = 1 THEN 'Total'
      ELSE category
    END AS category,
    ROUND(
      CASE 
        WHEN SUM(cnos_ya) = 0 THEN NULL
        ELSE 100.0 * SUM(cnos) / SUM(cnos_ya)
      END,
      1
    ) AS cnos_iya,
    ROUND(
      SUM(cnos) / 150.1 / 1000.0 / 1000.0,
      1
    ) AS cnos_mm
  FROM base_cnos
  WHERE month BETWEEN '2025M07' AND '2025M12'
  GROUP BY
    GROUPING SETS (
      (category),
      ()
    )
)

-- 2つの期間を UNION して1つのテーブルとして出力
SELECT
  term,
  category,
  cnos_iya,
  cnos_mm
FROM fy2525_1h

UNION ALL

SELECT
  term,
  category,
  cnos_iya,
  cnos_mm
FROM m0712

ORDER BY
  term,  -- まず期間ごとに並べる
  CASE 
    WHEN category = 'Total'           THEN 0
    WHEN category = 'Detergent'       THEN 1
    WHEN category = 'Fabric Enhancer' THEN 2
    WHEN category = 'Dish'            THEN 3
    WHEN category = 'Air'             THEN 4
    WHEN category = 'Baby'            THEN 5
    WHEN category = 'Fem'             THEN 6
    WHEN category = 'Hair'            THEN 7
    WHEN category = 'Shave'           THEN 8
    WHEN category = 'Oral'            THEN 9
    WHEN category = 'Braun'           THEN 10
    WHEN category = 'POC'             THEN 11
    ELSE 98
  END,
  category;

In [0]:
%sql

-- Team --

WITH base_cnos AS (
  -- ▼C-NOS のみを抽出し、金額は整数化しておく
  SELECT
    org2,
    org3,
    period,
    month,
    CAST(forecast_act_est AS INT) AS cnos,   -- 今年 C-NOS（整数）
    CAST(ya_actual        AS INT) AS cnos_ya -- 昨年 C-NOS（整数）
  FROM hive_metastore.japan_gold.cnos_vw
  WHERE account = 'C-NOS'
),

term_cnos AS (
  -- ▼対象期間に term ラベルを付与
  SELECT
    'FY2525 1H' AS term,
    org2,
    org3,
    cnos,
    cnos_ya
  FROM base_cnos
  WHERE period = 'FY2525 1H'

  UNION ALL

  SELECT
    '2025M07-12' AS term,
    org2,
    org3,
    cnos,
    cnos_ya
  FROM base_cnos
  WHERE month BETWEEN '2025M07' AND '2025M12'
),

summary AS (
  -- ▼各 term ごとに、Total / Div 2 / D-Com / Electro を一括で集計
  --   ・Div 2 は DP/MHC/WS（org3 ベース）の合計とみなす
  --   ・Div 1 は後で「残差」として計算する
  SELECT
    term,
    -- Total（全体）
    SUM(cnos)    AS total_cnos,
    SUM(cnos_ya) AS total_cnos_ya,

    -- Div 2 = DP + MHC + WS（org3 ベース）
    SUM(CASE WHEN org3 IN (
          'Drug/Pharmacy Team',
          'Mass & Home Center Team',
          'Wholesale team'
        )
        THEN cnos END)    AS div2_cnos,
    SUM(CASE WHEN org3 IN (
          'Drug/Pharmacy Team',
          'Mass & Home Center Team',
          'Wholesale team'
        )
        THEN cnos_ya END) AS div2_cnos_ya,

    -- D-Com（org2 ベース）
    SUM(CASE WHEN org2 = 'D-Com'
        THEN cnos END)    AS dcom_cnos,
    SUM(CASE WHEN org2 = 'D-Com'
        THEN cnos_ya END) AS dcom_cnos_ya,

    -- Electro（org2 ベース）
    SUM(CASE WHEN org2 = 'Electro'
        THEN cnos END)    AS electro_cnos,
    SUM(CASE WHEN org2 = 'Electro'
        THEN cnos_ya END) AS electro_cnos_ya
  FROM term_cnos
  GROUP BY term
),

detail AS (
  -- ▼DP / MHC / WS / D-Com / Electro の個別行（内訳）を集計
  --   ※ D-Com / Electro は summary でも使っているが、
  --     ここでは「行として出すため」に再度集計している
  SELECT
    term,
    'DP' AS team,
    SUM(cnos)    AS sum_cnos,
    SUM(cnos_ya) AS sum_cnos_ya
  FROM term_cnos
  WHERE org3 = 'Drug/Pharmacy Team'
  GROUP BY term

  UNION ALL

  SELECT
    term,
    'MHC' AS team,
    SUM(cnos)    AS sum_cnos,
    SUM(cnos_ya) AS sum_cnos_ya
  FROM term_cnos
  WHERE org3 = 'Mass & Home Center Team'
  GROUP BY term

  UNION ALL

  SELECT
    term,
    'WS' AS team,
    SUM(cnos)    AS sum_cnos,
    SUM(cnos_ya) AS sum_cnos_ya
  FROM term_cnos
  WHERE org3 = 'Wholesale team'
  GROUP BY term

  UNION ALL

  SELECT
    term,
    'D-Com' AS team,
    SUM(cnos)    AS sum_cnos,
    SUM(cnos_ya) AS sum_cnos_ya
  FROM term_cnos
  WHERE org2 = 'D-Com'
  GROUP BY term

  UNION ALL

  SELECT
    term,
    'Electro' AS team,
    SUM(cnos)    AS sum_cnos,
    SUM(cnos_ya) AS sum_cnos_ya
  FROM term_cnos
  WHERE org2 = 'Electro'
  GROUP BY term
),

node_agg AS (
  -- ▼Total / Div 1 / Div 2 と detail（DP/MHC/WS/D-Com/Electro）を一つのテーブルにまとめる
  --   Div 1 は「Total - Div2 - D-Com - Electro」で算出（残差）
  SELECT
    term,
    'Total' AS team,
    total_cnos    AS sum_cnos,
    total_cnos_ya AS sum_cnos_ya
  FROM summary

  UNION ALL

  SELECT
    term,
    'Div 2' AS team,
    div2_cnos    AS sum_cnos,
    div2_cnos_ya AS sum_cnos_ya
  FROM summary

  UNION ALL

  SELECT
    term,
    'Div 1' AS team,
    -- Div 1 = Total - Div 2 - D-Com - Electro（今年分）
    (total_cnos
     - COALESCE(div2_cnos, 0)
     - COALESCE(dcom_cnos, 0)
     - COALESCE(electro_cnos, 0)
    ) AS sum_cnos,
    -- Div 1 = Total - Div 2 - D-Com - Electro（昨年分）
    (total_cnos_ya
     - COALESCE(div2_cnos_ya, 0)
     - COALESCE(dcom_cnos_ya, 0)
     - COALESCE(electro_cnos_ya, 0)
    ) AS sum_cnos_ya
  FROM summary

  UNION ALL

  -- DP / MHC / WS / D-Com / Electro の個別行
  SELECT
    term,
    team,
    sum_cnos,
    sum_cnos_ya
  FROM detail
)

-- ▼最終出力：IYA と $MM を計算し、指定順に並べる
SELECT
  term,
  team,
  -- C-NOS IYA：今年 ÷ 昨年 × 100（小数第1位）
  ROUND(
    CASE
      WHEN sum_cnos_ya IS NULL OR sum_cnos_ya = 0 THEN NULL
      ELSE 100.0 * sum_cnos / sum_cnos_ya
    END,
    1
  ) AS cnos_iya,
  -- C-NOS $MM：cnos / 150.1 / 1000 / 1000（小数第1位）
  ROUND(
    CASE
      WHEN sum_cnos IS NULL THEN NULL
      ELSE sum_cnos / 150.1 / 1000.0 / 1000.0
    END,
    1
  ) AS cnos_mm
FROM node_agg
ORDER BY
  term,
  CASE
    -- 表示順：Total → Div 1 → Div 2 → DP → MHC → WS → D-Com → Electro
    WHEN team = 'Total'   THEN 1
    WHEN team = 'Div 1'   THEN 2
    WHEN team = 'Div 2'   THEN 3
    WHEN team = 'DP'      THEN 4
    WHEN team = 'MHC'     THEN 5
    WHEN team = 'WS'      THEN 6
    WHEN team = 'D-Com'   THEN 7
    WHEN team = 'Electro' THEN 8
    ELSE 99
  END,
  team;

In [0]:
%sql
-- ------------------------------------------------------------
-- 粒度：
--   term × customer_label
-- customer_label には以下を縦に並べる（同じ列）：
--   Div 1（17社合計）
--   Welcia, Cainz, ..., Sunday（各社）
--   D-Com（Amazon+Rakuten合計）
--   Amazon, Rakuten（各社）
--
-- 指標：
--   C-NOS IYA（今年÷昨年×100, 小数1桁）
--   C-NOS $MM（今年 C-NOS/150.1/1000/1000, 小数1桁）
--
-- 前提：
--   Div 1 = org2 = 'Division 1'
--   D-Com = org2 = 'D-Com'
-- ------------------------------------------------------------

WITH base_cnos AS (
  -- ▼C-NOS のみを抽出し、金額を整数化
  SELECT
    org2,
    customer,
    period,
    month,
    CAST(forecast_act_est AS INT) AS cnos,   -- 今年 C-NOS（整数）
    CAST(ya_actual        AS INT) AS cnos_ya -- 昨年 C-NOS（整数）
  FROM hive_metastore.japan_gold.cnos_vw
  WHERE account = 'C-NOS'
),

term_cnos AS (
  -- ▼対象期間に term ラベルを付与
  SELECT
    'FY2525 1H' AS term,
    org2,
    customer,
    cnos,
    cnos_ya
  FROM base_cnos
  WHERE period = 'FY2525 1H'

  UNION ALL

  SELECT
    '2025M07-12' AS term,
    org2,
    customer,
    cnos,
    cnos_ya
  FROM base_cnos
  WHERE month BETWEEN '2025M07' AND '2025M12'
),

filtered AS (
  -- ▼org2 と customer から Div 1 / D-Com を判定し、対象顧客だけに絞り込み
  SELECT
    term,
    CASE
      -- Div 1 顧客（org2 = Division 1）
      WHEN org2 = 'Division 1' AND customer IN (
        'FN-WELCIA YAKKYOKU',
        'FN-CAINZ(C003)',
        'FN-MATSUKIYO_COCOKARA (B008)',
        'FN-SUGI_YAKKYOKU(I015)',
        'FN-AEON_GROUP(JUSCOJ4)(C005)',
        'FN-COSMOS_YAKUHIN(W030)',
        'FN-COSTCO(A062)',
        'FN-TSURUHA_HOKK+TOHO(E030)',
        'FN-DONKIHOTE(A094)',
        'FN-SUN_DRUG(D005)',
        'FN-TGN',
        'FN-KYORINDO(H011)',
        'FN-LADY_YAKKYOKU(K067)',
        'FN-KUSURINO FUKUTARO (G043)',
        'FN-ELEVEN(L002)',
        'FN-UNY(B009)',
        'FN-SUNDAY(E015)'
      ) THEN 'Div 1'
      -- D-Com 顧客（org2 = D-Com）
      WHEN org2 = 'D-Com' AND customer IN (
        'FN-AMAZON JAPAN',
        'FN-RAKUTEN'
      ) THEN 'D-Com'
      ELSE NULL
    END AS division,          -- Div 1 / D-Com
    customer,
    cnos,
    cnos_ya
  FROM term_cnos
  WHERE customer IN (
    -- Div 1 顧客
    'FN-WELCIA YAKKYOKU',
    'FN-CAINZ(C003)',
    'FN-MATSUKIYO_COCOKARA (B008)',
    'FN-SUGI_YAKKYOKU(I015)',
    'FN-AEON_GROUP(JUSCOJ4)(C005)',
    'FN-COSMOS_YAKUHIN(W030)',
    'FN-COSTCO(A062)',
    'FN-TSURUHA_HOKK+TOHO(E030)',
    'FN-DONKIHOTE(A094)',
    'FN-SUN_DRUG(D005)',
    'FN-TGN',
    'FN-KYORINDO(H011)',
    'FN-LADY_YAKKYOKU(K067)',
    'FN-KUSURINO FUKUTARO (G043)',
    'FN-ELEVEN(L002)',
    'FN-UNY(B009)',
    'FN-SUNDAY(E015)',
    -- D-Com 顧客
    'FN-AMAZON JAPAN',
    'FN-RAKUTEN'
  )
),

cust_agg AS (
  -- ▼term × division × customer ごとに C-NOS を集計（顧客粒度）
  SELECT
    term,
    division,
    customer,
    SUM(cnos)    AS sum_cnos,
    SUM(cnos_ya) AS sum_cnos_ya
  FROM filtered
  WHERE division IS NOT NULL
  GROUP BY
    term,
    division,
    customer
),

div_agg AS (
  -- ▼Div 1 / D-Com の合計行（17社合計 / 2社合計）
  SELECT
    term,
    division,
    SUM(sum_cnos)    AS sum_cnos,
    SUM(sum_cnos_ya) AS sum_cnos_ya
  FROM cust_agg
  GROUP BY
    term,
    division
),

labeled AS (
  -- ▼Div 行と顧客行を「同じ列（customer_label）」に並べる
  SELECT
    term,
    -- Div 1・D-Com の合計行
    CASE
      WHEN division = 'Div 1' THEN 'Div 1'
      WHEN division = 'D-Com' THEN 'D-Com'
      ELSE division
    END AS customer_label,
    sum_cnos,
    sum_cnos_ya
  FROM div_agg

  UNION ALL

  SELECT
    term,
    -- 顧客行：短い名前に変換
    CASE customer
      WHEN 'FN-WELCIA YAKKYOKU'                 THEN ' Welcia'
      WHEN 'FN-CAINZ(C003)'                     THEN ' Cainz'
      WHEN 'FN-MATSUKIYO_COCOKARA (B008)'       THEN ' MCC'
      WHEN 'FN-SUGI_YAKKYOKU(I015)'             THEN ' Sugi'
      WHEN 'FN-AEON_GROUP(JUSCOJ4)(C005)'       THEN ' Aeon'
      WHEN 'FN-COSMOS_YAKUHIN(W030)'            THEN ' Cosmos'
      WHEN 'FN-COSTCO(A062)'                    THEN ' Costco'
      WHEN 'FN-TSURUHA_HOKK+TOHO(E030)'         THEN ' Tsuruha'
      WHEN 'FN-DONKIHOTE(A094)'                 THEN ' Donki'
      WHEN 'FN-SUN_DRUG(D005)'                  THEN ' Sundrug'
      WHEN 'FN-TGN'                             THEN ' TGN'
      WHEN 'FN-KYORINDO(H011)'                  THEN ' Kyorindo'
      WHEN 'FN-LADY_YAKKYOKU(K067)'             THEN ' Lady'
      WHEN 'FN-KUSURINO FUKUTARO (G043)'        THEN ' Fukutaro'
      WHEN 'FN-ELEVEN(L002)'                    THEN ' Eleven'
      WHEN 'FN-UNY(B009)'                       THEN ' Uny'
      WHEN 'FN-SUNDAY(E015)'                    THEN ' Sunday'
      WHEN 'FN-AMAZON JAPAN'                    THEN ' Amazon'
      WHEN 'FN-RAKUTEN'                         THEN ' Rakuten'
      ELSE customer
    END AS customer_label,
    sum_cnos,
    sum_cnos_ya
  FROM cust_agg
)

-- ▼最終出力：customer_label 1列で、Div1/顧客/D-Com/顧客を縦に並べる
SELECT
  term,
  customer_label AS customer,
  -- C-NOS IYA：今年 ÷ 昨年 × 100（小数第1位）
  ROUND(
    CASE
      WHEN sum_cnos_ya = 0 THEN NULL
      ELSE 100.0 * sum_cnos / sum_cnos_ya
    END,
    1
  ) AS cnos_iya,
  -- C-NOS $MM：cnos / 150.1 / 1000 / 1000（小数第1位）
  ROUND(
    sum_cnos / 150.1 / 1000.0 / 1000.0,
    1
  ) AS cnos_mm
FROM labeled
ORDER BY
  term,
  CASE
    -- 表示順：Div 1 → Div 1 顧客 → D-Com → D-Com 顧客
    WHEN customer = 'Div 1'   THEN 1
    WHEN customer = ' Welcia'  THEN 2
    WHEN customer = ' Cainz'   THEN 3
    WHEN customer = ' MCC'     THEN 4
    WHEN customer = ' Sugi'    THEN 5
    WHEN customer = ' Aeon'    THEN 6
    WHEN customer = ' Cosmos'  THEN 7
    WHEN customer = ' Costco'  THEN 8
    WHEN customer = ' Tsuruha' THEN 9
    WHEN customer = ' Donki'   THEN 10
    WHEN customer = ' Sundrug' THEN 11
    WHEN customer = ' TGN'     THEN 12
    WHEN customer = ' Kyorindo'THEN 13
    WHEN customer = ' Lady'    THEN 14
    WHEN customer = ' Fukutaro'THEN 15
    WHEN customer = ' Eleven'  THEN 16
    WHEN customer = ' Uny'     THEN 17
    WHEN customer = ' Sunday'  THEN 18
    WHEN customer = 'D-Com'   THEN 19
    WHEN customer = ' Amazon'  THEN 20
    WHEN customer = ' Rakuten' THEN 21
    ELSE 99
  END,
  customer;